# Imports

In [1]:
import numpy as np
import pandas as pd
from custom_modules.helper_functions import * 

# Read Data

In [2]:
df_curr = pd.read_csv("..\\Data\\Refined Data\\Currencies\\currencies.csv",
                      parse_dates=[1])

# Define Returns

In [3]:
# Take lags of forward rates and spot mid rates, for use in defining
# carry trade excess returns and changes in spot rate
df_curr[["fb_m1", "fm_m1", "fa_m1", "sm_m1"]] = (
    df_curr.groupby("curr")[["for_bid", "for_mid", "for_ask", "spot_mid"]]
           .shift(1)
)

# Define the change in spot rates in logs
df_curr["del_spot"] = np.log(df_curr.spot_mid) - np.log(df_curr.sm_m1)

# Define the forward discount
df_curr["fwd_discount"] = np.log(df_curr.fm_m1) - np.log(df_curr.sm_m1)

# Define log carry trade return
df_curr["ret"] = df_curr.fwd_discount - df_curr.del_spot

# Define long and short net log excess returns in month t similarly, but
# using bid and ask rates
df_curr["ret_net_lg"] = np.log(df_curr.fb_m1) - np.log(df_curr.spot_ask)
df_curr["ret_net_sh"] = -np.log(df_curr.fa_m1) + np.log(df_curr.spot_bid)

# Group by forward discount

In [4]:
# Create cut-offs for 6 quantiles
group_cutoffs = (
    df_curr.groupby("date").fwd_discount.quantile([1/6, 2/6, 3/6, 4/6, 5/6])
           .to_frame().reset_index()
           .pivot(index="date", columns="level_1", values="fwd_discount")
           .reset_index()
)

group_cutoffs.columns = ["date", "fd_1", "fd_2", "fd_3", "fd_4", "fd_5"]

# Merge cut-offs back in
df_curr_grouped = (
    df_curr.merge(group_cutoffs, on="date").sort_values(by=["date", "curr"])
)

# Calculate quantile number at time t-1
df_curr_grouped["group_num"] = (
    np.where(
        df_curr_grouped.fwd_discount <= df_curr_grouped.fd_1, 1,
        np.where(
            df_curr_grouped.fwd_discount <= df_curr_grouped.fd_2, 2,
            np.where(
                df_curr_grouped.fwd_discount <= df_curr_grouped.fd_3, 3,
                np.where(
                    df_curr_grouped.fwd_discount <= df_curr_grouped.fd_4, 4, 
                    np.where(
                        df_curr_grouped.fwd_discount <= df_curr_grouped.fd_5,
                        5, 6
                    )
                )
            )
        )
    )
)

# Create grouped portfolios

In [5]:
# Take logs of necessary portfolio average observations
df_curr_ports = (
    df_curr_grouped.groupby(["date", "group_num"])\
                   .agg(del_spot=("del_spot", "mean"),
                        fwd_discount=("fwd_discount", "mean"),
                        ret=("ret", "mean"),
                        ret_net_lg=("ret_net_lg", "mean"),
                        ret_net_sh=("ret_net_sh", "mean"))
                   .reset_index()
)

In [6]:
# Fix net returns
df_curr_ports["ret_net"] = np.where(df_curr_ports.group_num == 1,
                                    df_curr_ports.ret_net_sh,
                                    df_curr_ports.ret_net_lg)

df_curr_ports.drop(["ret_net_lg", "ret_net_sh"], axis=1, inplace=True)

# Inspect grouped portfolios

In [7]:
# Create a summary DataFrame similar to the one in LRV (2011)
df_summary = pd.DataFrame(columns=["1", "2", "3", "4", "5", "6"])

# Change in spot rate in logs
df_summary.loc["Δs (mean)", :] = (df_curr_ports.groupby("group_num")
                                               .del_spot.mean()
                                               * 1200).values
df_summary.loc["Δs (std)", :] = (df_curr_ports.groupby("group_num")
                                              .del_spot.std()
                                              * np.sqrt(12)*100).values

# Forward discount
df_summary.loc["f-s (mean)", :] = (df_curr_ports.groupby("group_num")
                                                .fwd_discount.mean()
                                                * 1200).values
df_summary.loc["f-s (std)", :] = (df_curr_ports.groupby("group_num")
                                               .fwd_discount.std()
                                               * np.sqrt(12)*100).values

# Excess return
df_summary.loc["rx (mean)", :] = (df_curr_ports.groupby("group_num")
                                               .ret.mean()
                                               * 1200).values
df_summary.loc["rx (std)", :] = (df_curr_ports.groupby("group_num")
                                              .ret.std()
                                              * np.sqrt(12)*100).values
df_summary.loc["rx (SR)", :] = (
    df_summary.loc["rx (mean)", :].div(df_summary.loc["rx (std)", :])
)

# Excess net return
df_summary.loc["rx_net (mean)", :] = (df_curr_ports.groupby("group_num")
                                                   .ret_net.mean()
                                                   * 1200).values
df_summary.loc["rx_net (std)", :] = (df_curr_ports.groupby("group_num")
                                                  .ret_net.std()
                                                  * np.sqrt(12)*100).values
df_summary.loc["rx_net (SR)", :] = (
    df_summary.loc["rx_net (mean)", :].div(df_summary.loc["rx_net (std)", :])
)

In [8]:
df_summary

,1,2,3,4,5,6
Δs (mean),-0.644963,-0.32804,-0.430232,-1.478847,1.338368,5.264857
Δs (std),7.29209,6.448507,6.90521,7.57225,8.481703,10.075641
f-s (mean),-2.256781,-0.773654,0.361987,1.570437,3.417412,10.035883
f-s (std),0.502067,0.424049,0.386002,0.417156,0.548351,1.864167
rx (mean),-1.611818,-0.445614,0.792219,3.049199,2.079043,4.769104
rx (std),7.307233,6.455874,6.934276,7.61099,8.467846,10.075066
rx (SR),-0.220578,-0.069025,0.114247,0.400631,0.245522,0.473357
rx_net (mean),0.660692,-1.31092,-0.430114,1.696714,0.562494,2.357677
rx_net (std),7.315014,6.452888,6.907007,7.581305,8.496005,10.039797
rx_net (SR),0.09032,-0.203152,-0.062272,0.223802,0.066207,0.234833


# Calculate Monthly Factors

In [11]:
# Aggregate factor data
df_curr_factors = (
    df_curr_ports.sort_values(by=["date", "group_num"]).groupby("date")
                 .agg(rx=("ret", "mean"),
                      hml_high=("ret", "last"),
                      hml_low=("ret", "first"),
                      rx_net=("ret_net", "mean"),
                      hml_high_net=("ret_net", "last"),
                      hml_low_net=("ret_net", "first"),
                      dollar=("del_spot", "mean"),
                      carry_high=("del_spot", "last"),
                      carry_low=("del_spot", "first"))
)

# Define HML_FX and Carry
df_curr_factors["hml_fx"] = df_curr_factors.hml_high - df_curr_factors.hml_low
df_curr_factors["hml_fx_net"] = (df_curr_factors.hml_high_net
                                 - df_curr_factors.hml_low_net)

df_curr_factors["carry"] = (df_curr_factors.carry_high
                            - df_curr_factors.carry_low)

# Choose columns and convert returns to percentage points
df_curr_factors = (
    df_curr_factors.copy().loc[:, ["hml_fx", "rx", "hml_fx_net",
                                   "rx_net", "carry", "dollar"]] * 100
)

# Save Factors

In [13]:
df_curr_factors.to_csv("..\\Data\\Refined Data\\Currencies\\curr_factors.csv")